In [1]:
# pip install tensorflow opencv-python-headless scikit-learn

In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.preprocessing import LabelEncoder

# Define the labels
LABELS = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 
          'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 
          'W', 'X', 'Y', 'Z', 'del', 'nothing', 'space']

# Encode the labels
label_encoder = LabelEncoder()
label_encoder.fit(LABELS)

# Try to load the pre-trained model
try:
    model = load_model('sign_language_model.h5')  # Replace with your model file
    print("Model loaded successfully!")
except Exception as e:
    print(f"No pre-trained model found. Creating a new model... Error: {e}")
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),  # Ensures proper input to Dense layer
         # Matches Flatten output
        Dense(1024, activation='relu'),
        Dense(len(LABELS), activation='softmax')  # Output layer for classification
    ])
    # Compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Debugging: Print layer outputs to ensure compatibility
sample_input = np.random.random((1, 64, 64, 3))
for layer in model.layers:
    sample_input = layer(sample_input)
    print(f"After layer {layer.name}, shape: {sample_input.shape}")

# Function to preprocess frames
def preprocess_frame(frame):
    """
    Resize, normalize, and prepare the frame for model prediction.
    """
    resized = cv2.resize(frame, (64, 64), interpolation=cv2.INTER_AREA)
    normalized = resized.astype('float32') / 255.0
    return np.expand_dims(normalized, axis=0)  # Add batch dimension

# Initialize webcam
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Retrying webcam initialization...")
    cap.open(0)
if not cap.isOpened():
    print("Error: Unable to access webcam. Exiting.")
    exit()

print("Press 'q' to quit.")

# Main loop
while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Could not read frame.")
        break

    # Mirror the frame for convenience
    frame = cv2.flip(frame, 1)

    # Detect hand (optional, can add hand detection logic here)
    region_of_interest = frame

    # Preprocess the frame
    processed_frame = preprocess_frame(region_of_interest)

    # Predict the sign language gesture
    prediction = model.predict(processed_frame)
    predicted_class = np.argmax(prediction)
    translated_sign = label_encoder.inverse_transform([predicted_class])[0]

    # Display the prediction
    cv2.putText(frame, f"Predicted Sign: {translated_sign}", (10, 40),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Show the video feed
    cv2.imshow("Sign Language Recognition", frame)

    # Exit on pressing 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close all windows
cap.release()
cv2.destroyAllWindows()
